## Import the relevant libraries

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

C:\Users\Acer\Documents\anaconda3\envs\py3-TF2.0\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading and preprocessing data

In [2]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info=True, as_supervised=True)

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

#storing amount of 10% validation data from training data
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

#storing number of test samples
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [4]:
#function used to scale training, validation and test data
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image = float(image/255)
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

scaled_test_data = mnist_test.map(scale)

#Shuffling the data
buffer_size = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)

#Extracting validation and training data from the suffled data
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


In [5]:
#Batching the data for mini-batch gradient descent

Batch_size = 100

train_data = train_data.batch(Batch_size)
validation_data = validation_data.batch(num_validation_samples)
scaled_test_data = scaled_test_data.batch(num_test_samples)

validation_inputs , validation_targets = next(iter(validation_data))

## MODEL

### Outlining the model

In [12]:
# Defining sizes of input, hidden and output layers
input_size = 784 # 28 * 28 pixels = 784 pixels
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
    tf.keras.layers.Dense(output_size, activation = 'softmax'),
])

### Choosing the optimizer and loss function

In [13]:
model.compile(optimizer = 'adam' , loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

### Training the model

In [14]:
num_epochs = 5
model.fit(train_data, epochs = num_epochs, validation_data = (validation_inputs, validation_targets), verbose = 2)

Epoch 1/5
540/540 - 2s - loss: 0.2449 - accuracy: 0.9272 - val_loss: 0.1197 - val_accuracy: 0.9627 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 1s - loss: 0.0962 - accuracy: 0.9701 - val_loss: 0.0817 - val_accuracy: 0.9718 - 1s/epoch - 3ms/step
Epoch 3/5
540/540 - 1s - loss: 0.0628 - accuracy: 0.9795 - val_loss: 0.0531 - val_accuracy: 0.9822 - 1s/epoch - 3ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0493 - accuracy: 0.9838 - val_loss: 0.0571 - val_accuracy: 0.9803 - 2s/epoch - 3ms/step
Epoch 5/5
540/540 - 1s - loss: 0.0370 - accuracy: 0.9880 - val_loss: 0.0502 - val_accuracy: 0.9840 - 1s/epoch - 3ms/step


## Testing the Model

In [16]:
test_loss, test_accuracy = model.evaluate(scaled_test_data)

1/1 [==============================] - 0s 271ms/step - loss: 0.0828 - accuracy: 0.9750


In [17]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.50%
